In [ ]:
!pip install transformers
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.5 MB/s eta 0:00:00
TF version:  2.12.0
Hub version:  0.13.0


In [ ]:
import pandas as pd

In [ ]:
!pip install datasets
from datasets import load_dataset
from datasets import DatasetDict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.2 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('label.csv')

In [ ]:
labels = df[['TEXT']]

In [ ]:
labels['label'] = df['label'].apply(lambda x: x-1)

<ipython-input-6-9811cead2fac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['label'] = df['label'].apply(lambda x: x-1)


In [ ]:
labels.groupby('label').count()

,TEXT
label,
0,78
1,146
2,112
3,1003
4,1061


In [ ]:
labels.to_csv('finegrained_labels_processed.csv')

In [ ]:
dataset_train_eval = (load_dataset("csv", data_files="finegrained_labels_processed.csv", split = "train").train_test_split(test_size=0.3))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bc4c714504b24d75/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [ ]:
dataset_test_eval = dataset_train_eval["test"].train_test_split(test_size=0.5)

In [ ]:
train_test_valid_dataset = DatasetDict({
    'train': dataset_train_eval['train'],
    'test': dataset_test_eval['test'],
    'valid': dataset_test_eval['train']})

In [ ]:
train_test_valid_dataset["train"]

Dataset({
    features: ['Unnamed: 0', 'TEXT', 'label'],
    num_rows: 1680
})

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_function(examples):
    return tokenizer(examples["TEXT"], padding="max_length", truncation=True, max_length=142)


tokenized_datasets_train = train_test_valid_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_labelled_test = train_test_valid_dataset['test'].map(tokenize_function, batched=True)
tokenized_datasets_val = train_test_valid_dataset['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=5,ignore_mismatched_sizes=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [ ]:
!pip install evaluate
import numpy as np
import evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 230.2 kB/s eta 0:00:00


In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


In [ ]:
import evaluate

metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision", average="weighted")
recall_metric = evaluate.load("recall", average="weighted")
f1_metric = evaluate.load("f1", average="weighted")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.from_numpy(logits)
    labels = torch.from_numpy(labels)
    predictions = torch.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1-score": f1}

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", logging_steps = 88, evaluation_strategy="epoch")

In [ ]:
model.config.num_labels

5

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 5 labels with different weights)
        loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor([13.7, 6.8, 9.1, 1.0, 0.94])).to(device)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1-score
1,0.667600,1.699995,0.716667,0.727112,0.716667,0.715842
2,0.413600,1.398665,0.694444,0.696623,0.694444,0.691798
3,0.252000,1.941950,0.708333,0.713525,0.708333,0.707606


TrainOutput(global_step=630, training_loss=0.4420374766228691, metrics={'train_runtime': 171.9704, 'train_samples_per_second': 29.307, 'train_steps_per_second': 3.663, 'total_flos': 367789828512960.0, 'train_loss': 0.4420374766228691, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets_labelled_test)

In [ ]:
trainer.evaluate()

{'eval_loss': 1.9419504404067993,
 'eval_accuracy': 0.7083333333333334,
 'eval_precision': 0.713524845610759,
 'eval_recall': 0.7083333333333334,
 'eval_f1-score': 0.7076056030640244,
 'eval_runtime': 5.2889,
 'eval_samples_per_second': 68.067,
 'eval_steps_per_second': 8.508,
 'epoch': 3.0}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
trainer.save_model("/content/gdrive/MyDrive/trained_model_finegrained")